In [4]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['/Users/philipmassey/PycharmProjects/stock_market'])
import portfolio as pf
import pandas as pd
import yfinance as yf
import calendar
from datetime import datetime
df_port = pf.getPortfolios()
symbols = list(df_port.index.values)


Python 3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 16:52:21) 
[Clang 6.0 (clang-600.0.57)] on darwin


In [5]:
def getStockPickleDirectory():
    path = '/Users/philipmassey/PycharmProjects/stock_market/market_data/stock_pickles'
    return path
def getPickleName(rdate):
    ppath = getStockPickleDirectory()
    return ppath+'/'+rdate+'.pk'

def getNDaysAgo(ndays):
    now = datetime.now()
    start = now - pd.tseries.offsets.BusinessDay(n=(ndays+1))
    end = now - pd.tseries.offsets.BusinessDay(n=ndays)
    start = '{:%Y-%m-%d}'.format(start)
    end = '{:%Y-%m-%d}'.format(end)
    return start,end

def getRowNAgo(symbols, ndays):
    if ndays == 0:
        dfDaysAgo = yf.download(tickers = symbols,period = "1d",interval = "1d",group_by = 'column',auto_adjust = True,prepost = False,threads = True)
        dfDaysAgo = dfDaysAgo.tail(1)
    else:
        start, end = getNDaysAgo(ndays)
        dfDaysAgo = yf.download(tickers = symbols,interval = "1d",start=start,end=end,group_by = 'column',auto_adjust = True,prepost = False,threads = True)
        dfDaysAgo = dfDaysAgo.head(1)
    return dfDaysAgo[['Close','Volume']]

def getDescriptiveDate(dfRow):
    date = pd.to_datetime(dfRow.index.values[0])
    return calendar.day_name[date.weekday()]+' '+'{:%Y-%m-%d}'.format(date)
def getRowYmdDate(dfRow):
    rowdate =  pd.to_datetime(dfRow.index.values[0])
    return '{:%Y-%m-%d}'.format(rowdate)


In [6]:
def pickleStock(ndays):
    print(ndays)
    dfnrow = getRowNAgo(symbols,ndays)
    if len(dfnrow.index.values) == 0:
        print('{} {}'.format('Failed',ndays))
        exit()
    rdate = getRowYmdDate(dfnrow)
    dfnrow.to_pickle(getPickleName(rdate))
    return '{} {}'.format('Pickled',ndays)
#[pickleStock(ndays) for ndays in range(185,186)]

In [9]:
def getStockPickle(ndays):
    start,end = getNDaysAgo(ndays)
    print(start,end)
    pk_name = getPickleName(start)
    try:
        dfNDaysAgo = pd.read_pickle(pk_name)
    except FileNotFoundError as e:
        print(e,ndays,start)
        return
    return dfNDaysAgo

ndays = 190
getStockPickle(ndays)
    

2020-02-07 2020-02-10
[Errno 2] No such file or directory: '/Users/philipmassey/PycharmProjects/stock_market/market_data/stock_pickles/2020-02-07.pk' 190 2020-02-07


In [ ]:
data = pd.read_pickle(getPickleName(rdate))
data